In [1]:
"""Anomaly Detection Example"""
from __future__ import print_function
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
from matplotlib import pyplot as plt
%matplotlib inline
import os
import sys
import argparse
import math
from collections import Counter
import ipaddress
import seaborn as sns
# Third Party Imports
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.cluster import KMeans
from sklearn.externals import joblib
# Local imports
from zat import log_to_dataframe
from zat import dataframe_to_matrix
import numpy as np
pd.set_option('display.max_rows', 500)
import swifter


C:\Users\atti-001\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


## IESCO 'Conn' data is being loaded
##### Total length of data ist wo million six hundred sixty-two thousand eight hundred sixty-three records

In [2]:
pd.set_option('display.width', 1000)
bro_log = 'http'
bro_df = pd.read_csv('../../../IESCO_complete_dec_may/preprocessed_csv/conn_full.csv')

In [3]:
bro_df.head(5)

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,...,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents
0,2020-03-31 19:59:54.307151079,C8TyRZDMgWpC3Vevf,39.41.241.162,36186,172.16.2.211,8081,tcp,http,0.305089,140,...,SF,F,F,0,ShADadfF,6,460,5,405,(empty)
1,2020-03-31 19:59:56.170258045,CIGXkd4ajdDaSIaLcd,39.40.41.30,35130,172.16.2.211,8081,tcp,http,0.241368,140,...,SF,F,F,0,ShADadfF,5,408,5,405,(empty)
2,2020-03-31 19:59:56.478173971,Cil8w34oLPNQ8mrXWd,39.50.98.217,45948,172.16.2.211,8081,tcp,http,0.241572,140,...,SF,F,F,0,ShADadfF,5,408,5,405,(empty)
3,2020-03-31 19:59:56.905885935,C3YWKFgSzSfTUUvz,39.41.160.14,43330,172.16.2.211,8081,tcp,http,0.265524,140,...,SF,F,F,0,ShADadfF,5,408,5,405,(empty)
4,2020-03-31 19:59:58.210148096,C3vir25kXkLv8MVL2,39.41.194.146,56366,172.16.2.211,8081,tcp,http,0.252428,140,...,SF,F,F,0,ShADadfF,5,408,5,405,(empty)


In [15]:
pd.concat([bro_df.sample(5000),bro_df[(bro_df['id.orig_h'] =='172.16.2.90') & (bro_df['id.resp_h'] == '52.216.129.43')]],axis=0).to_json('conn_test.json')

### Loading IP List & Mapping IPS

In [4]:
ip_info = pd.read_csv('../../IESCO_IP_LIST.csv').dropna()
hostnames = list(ip_info['Source'].values)
ips = list(ip_info['IP'].values)
d = {k:v for k, v in zip(ips,hostnames)}


In [5]:
ip_info.head(10)

,Source,IP
0,Vmware Vcenter,172.16.1.50
1,Vcenter server Appliance,172.16.1.50/ port# 5480
2,Vmware Vcenter- HA user,172.16.1.50
3,Vmware Vcenter- Veeam user,172.16.1.50
4,ESXI-1- Veeam user,172.16.1.51
5,ESXI-2- Veeam user,172.16.1.52
6,ESXI-3- Veeam user,172.16.1.53
7,ESXI-4- Veeam user,172.16.1.54
8,ESXI-5- Veeam user,172.16.1.55
10,iBMC,192.168.1.51


## Helping function

1. validate_ip
2. traffic_direction using RFC 1918 Logic
3. calculate entropy against domain names

In [6]:
def validate_ip(ip):
    try:
        ipaddress.ip_address(ip)
        return ip
    except:
        return np.nan

def traffic_direction(conn_row):
    # First try to use the local orig/resp fields
#     if conn_row.get('local_orig') and conn_row.get('local_resp'):
#         local_orig = conn_row['local_orig']
#         local_resp = conn_row['local_resp']
#     else:
        # Well we don't have local orig/resp fields so use RFC1918 logic
    local_orig = ipaddress.ip_address(conn_row['id.orig_h']).is_private
    local_resp = ipaddress.ip_address(conn_row['id.resp_h']).is_private

    # Determine north/south or internal traffic
    if (not local_orig) and local_resp:
        return 'incoming'
    if local_orig and not local_resp:
        return 'outgoing'

    # Neither host is in the allocated private ranges
    if ipaddress.ip_address(conn_row['id.orig_h']).is_multicast or \
       ipaddress.ip_address(conn_row['id.resp_h']).is_multicast:
        return 'multicast'

    # Both hosts are internal
    return 'internal'

def entropy(string):
    """Compute entropy on the string"""
    try:
        p, lns = Counter(string), float(len(string))
        return -sum(count/lns * math.log(count/lns, 2) for count in p.values())
    except Exception as e:
        return pd.np.nan

def ip_inv_embedding_ext(ip):
    tranformed = None
    try:
        tranformed = str(ipaddress.ip_address(ip))
        return tranformed
    except:
        return tranformed

def enrich_names(bro_df):
    bro_df['Hostname_orig'] = bro_df['id.orig_h'].swifter.apply(lambda x : d.get(x,np.nan))
    bro_df['Hostname_resp'] = bro_df['id.resp_h'].swifter.apply(lambda x : d.get(x,np.nan))
    return bro_df


### Preprocessing

Applied various preprocessing function to clean raw bro logs.

In [7]:
def preprocessing(bro_df):
    print('preprocessing....') 
    print('4. Cleaning Services.....')
    bro_df=bro_df[bro_df['service'].isin(['http','dns','dhcp','ssl','ssh'])]
    bro_df['orig_bytes'] = pd.to_numeric(bro_df['orig_bytes'],errors='coerce')
    bro_df['orig_bytes'] = bro_df['orig_bytes'].fillna(0)
    
    print('1. Cleaning/ Validating IP address.....')
    bro_df['id.orig_h'] = bro_df['id.orig_h'].swifter.apply(lambda x : validate_ip(x))
    bro_df['id.resp_h'] = bro_df['id.resp_h'].swifter.apply(lambda x : validate_ip(x))
    bro_df = bro_df[bro_df['id.orig_h'].notna() & bro_df['id.resp_h'].notna()]
    
    print('2. Assinging Names to IP address.....')
    bro_df['Hostname_orig'] = bro_df['id.orig_h'].swifter.apply(lambda x : d.get(x,np.nan))
    bro_df['Hostname_resp'] = bro_df['id.resp_h'].swifter.apply(lambda x : d.get(x,np.nan))

    print('3. Generating direction..........')
    bro_df['direction']=bro_df.swifter.apply(lambda row :traffic_direction(row) , axis = 1) 
   
 
    
    print('5. Cleansing resp bytes')
    bro_df['resp_bytes'] = pd.to_numeric(bro_df['resp_bytes'],errors='coerce')
    bro_df['resp_bytes'] = bro_df['resp_bytes'].fillna(0)
    
    print('6. Cleansing resp packets')
    bro_df['resp_pkts'] = pd.to_numeric(bro_df['resp_pkts'],errors='coerce')
    bro_df['resp_pkts'] = bro_df['resp_pkts'].fillna(0)
    
    print('7. Cleansing orig_ip_bytes ')
    bro_df['orig_ip_bytes'] = pd.to_numeric(bro_df['orig_ip_bytes'],errors='coerce')
    bro_df['orig_ip_bytes'] = bro_df['orig_ip_bytes'].fillna(0)

    print('8. Cleansing resp_ip_bytes ')
    bro_df['resp_ip_bytes'] = pd.to_numeric(bro_df['resp_ip_bytes'],errors='coerce')
    bro_df['resp_ip_bytes'] = bro_df['resp_ip_bytes'].fillna(0)

    print('9. Cleansing id.resp_p')
    bro_df['id.resp_p'] = pd.to_numeric(bro_df['id.resp_p'],errors='coerce')
    bro_df['id.resp_p'] = bro_df['id.resp_p'].fillna(0)
    
    print('10. Cleansing id.orig_p')
    bro_df['id.orig_p'] = pd.to_numeric(bro_df['id.resp_p'],errors='coerce')
    bro_df['id.orig_p'] = bro_df['id.resp_p'].fillna(0)
    
    print('11. Converting Duration to Seconds')
    bro_df['durationsec'] = pd.to_numeric(bro_df.duration,errors='coerce')
    bro_df['durationsec'] = bro_df['durationsec'].fillna(0)
    
    #bro_df['id.orig_h_e']=bro_df['id.orig_h'].apply(lambda x : ip_embedding_ext(x))
    return bro_df

### Apply Aggregation 

After preprocessing. We engineered 20 features from raw bro logs and aggregated them based on Source Address, Destination Address & 5 minutes time windows.


In [8]:
import pandas as pd
def getFeatures(df):
    window_size = 1000
    total_batch = int(len(df)/window_size)
    trainX = pd.DataFrame()
    # Loop over all batches
    for i in range(total_batch):
        n = min(window_size, len(df)-i*window_size)
        df_win = df[i*window_size:i*window_size+n]
        df_win['ts'] = pd.to_datetime(df_win['ts'])
        df_win.set_index(['SrcAddr','DstAddr','ts'],inplace=True,drop=True)
        group = df_win.groupby([pd.Grouper(level='SrcAddr'),pd.Grouper(level='DstAddr'),pd.Grouper(level='ts',freq='5T')])
        #df_win.groupby(['SrcAddr'])
        train_win = group.Sport.nunique()
        train_win = pd.DataFrame(train_win)
        #train_win['udest_ip'] = group.DstAddr.nunique()
        train_win['udest_port'] = group.Dport.nunique()
        train_win['netflows'] = group.Dport.count()
        train_win['uproto'] = group.Proto.nunique()
        train_win['TotDur'] = group.Dur.sum()
        train_win['minDur'] = group.Dur.min()
        train_win['maxDur'] = group.Dur.max()
        temp = group.Proto.value_counts().unstack().fillna(0)
        try:
            train_win['tcp'] = temp['tcp']
        except:
            train_win['tcp'] = group.Dport.count()*0
        try:
            train_win['udp'] = temp['udp']
        except:
             train_win['udp'] = group.Dport.count()*0
                
                
                
                
        temp = group.dir.value_counts().unstack().fillna(0)
        try:
            train_win['internal'] = temp['internal']
        except:
            train_win['internal'] = 0
        try:
            train_win['incoming'] = temp['incoming']
        except:
             train_win['incoming'] = 0
        try:
            train_win['outgoing'] = temp['outgoing']
        except:
             train_win['outgoing'] = 0
        
        temp = group.service.value_counts().unstack().fillna(0)
        try:
            train_win['http'] = temp['http']
        except:
            train_win['http'] = 0
        try:
            train_win['dns'] = temp['dns']
        except:
             train_win['dns'] = 0
        try:
            train_win['dhcp'] = temp['dhcp']
        except:
             train_win['dhcp'] = 0
        try:
            train_win['ssl'] = temp['ssl']
        except:
             train_win['ssl'] = 0
        try:
            train_win['ssh'] = temp['ssh']
        except:
             train_win['ssh'] = 0
                
                
        train_win['bytes'] = group.TotBytes.sum()
        train_win['packets'] = group.TotPkts.sum()
        #train_win['Label'] = (group.Label.sum()*1.0/group.Dport.count())> 0
        trainX = pd.concat([trainX, train_win], axis=0)
    return trainX.round(2)

In [9]:
%%time 
bro_df=preprocessing(bro_df)

preprocessing....
4. Cleaning Services.....
1. Cleaning/ Validating IP address.....



2. Assinging Names to IP address.....



3. Generating direction..........



5. Cleansing resp bytes
6. Cleansing resp packets
7. Cleansing orig_ip_bytes 
8. Cleansing resp_ip_bytes 
9. Cleansing id.resp_p
10. Cleansing id.orig_p
11. Converting Duration to Seconds
Wall time: 10min 21s


In [10]:
bro_df['TotPkts']= pd.to_numeric(bro_df['orig_pkts'],errors='coerce')+pd.to_numeric(bro_df['resp_pkts'],errors='coerce')
bro_df['TotBytes'] = pd.to_numeric(bro_df['resp_bytes'],errors='coerce')+pd.to_numeric(bro_df['orig_bytes'],errors='coerce')

In [11]:
df_test = bro_df[['ts','id.orig_h','uid','id.resp_h','proto','id.resp_p','id.orig_p','TotBytes','TotPkts','durationsec','direction','service']]
df_test.columns=['ts','SrcAddr','session_id','DstAddr','Proto','Dport','Sport','TotBytes','TotPkts','Dur','dir','service']

In [12]:
df_test.shape

(11042093, 12)

In [13]:
11042093%8

5

In [14]:
splits = np.split(df_test.iloc[df_test.shape[0] % 8:,:],8)

In [15]:
from joblib import delayed,Parallel
list_df = Parallel(n_jobs=8)(delayed(getFeatures)(i) for i in splits)
df_test = pd.concat(list_df, axis=0)

In [16]:
joblib.dump(df_test,'feature_set/con_featureset_v2.pkl')

['feature_set/con_featureset_v2.pkl']

In [17]:
df_test.head(5)

Sport  udest_port  netflows  uproto  TotDur  minDur  maxDur  tcp  udp  internal  incoming  outgoing  http  dns  dhcp  ssl  ssh  bytes  packets
SrcAddr    DstAddr     ts                                                                                                                                                                 
10.1.1.210 10.1.1.255  2020-03-31 19:55:00      1           1         1       1    1.51    1.51    1.51  0.0  1.0       1.0       0.0       0.0   0.0  1.0   0.0  0.0  0.0    300        6
10.1.1.23  224.0.0.251 2020-03-31 20:05:00      1           1         1       1    0.00    0.00    0.00  0.0  1.0       0.0       0.0       1.0   0.0  1.0   0.0  0.0  0.0     82        2
10.1.1.5   10.1.1.255  2020-03-31 19:55:00      1           1         1       1    1.52    1.52    1.52  0.0  1.0       1.0       0.0       0.0   0.0  1.0   0.0  0.0  0.0    300        6
                       2020-03-31 20:00:00      1           1         6       1    9.31    1.53    1.57  0.0  6.0       6.0       0.0       0.0   0.0  6.0   0.0  0.0  0.0   1800       36
                       2020-03-31 20:05:00      1           1         5       1   15.12    1.50    8.93  0.0  5.0       5.0       0.0       0.0   0.0  5.0   0.0  0.0  0.0   1800       36

In [18]:
bro_log ='conn'

In [19]:
if 'conn' in bro_log:
    log_type = 'conn'
    features =['Sport', 'udest_port', 'netflows', 'uproto', 'TotDur', 'minDur', 'maxDur', 'tcp', 'udp', 'internal', 'incoming', 'outgoing', 'http', 'dns', 'dhcp', 'ssl', 'ssh','bytes','packets']
    #features = ['durationsec', 'orig_bytes', 'id.resp_p', 'resp_bytes', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes','service','proto','direction']
elif 'dns' in bro_log:
    log_type = 'dns'
    features = ['Z', 'proto', 'qtype_name','id.orig_p','id.resp_p', 'query_length', 'answer_length', 'entropy']
else:
    print('This example only works with Zeek with http.log or dns.log files..')
    sys.exit(1)

# Create a Pandas dataframe from a Zeek log
try:
    log_to_df = log_to_dataframe.LogToDataFrame()
    #bro_df = pd.read_csv('http_Farrukh-Naveed-Anjum.csv')#log_to_df.create_dataframe(bro_log)
    #print(bro_df.head())
except IOError:
    print('Could not open or parse the specified logfile: %s' % bro_log)
    sys.exit(1)
print('Read in {:d} Rows...'.format(len(bro_df)))

# Using Pandas we can easily and efficiently compute additional data metrics
# Here we use the vectorized operations of Pandas/Numpy to compute query length
# We'll also compute entropy of the query
if log_type == 'dns':
    bro_df['query_length'] = bro_df['query'].str.len()
    bro_df['answer_length'] = bro_df['answers'].str.len()
    bro_df['entropy'] = bro_df['query'].map(lambda x: entropy(x))
# Use the zat DataframeToMatrix class
to_matrix = dataframe_to_matrix.DataFrameToMatrix()
bro_matrix = to_matrix.fit_transform(df_test[features])
print(bro_matrix.shape)

Read in 11042093 Rows...
Normalizing column Sport...
Normalizing column udest_port...
Normalizing column netflows...
Normalizing column uproto...
Normalizing column TotDur...
Normalizing column minDur...
Normalizing column maxDur...
Normalizing column tcp...
Normalizing column udp...
Normalizing column internal...
Normalizing column incoming...
Normalizing column outgoing...
Normalizing column http...
Normalizing column dns...
Normalizing column dhcp...
Normalizing column ssl...
Normalizing column ssh...
Normalizing column bytes...
Normalizing column packets...
(2079239, 19)


In [20]:
# Train/fit and Predict anomalous instances using the Isolation Forest model
odd_clf = IsolationForest(behaviour='new',contamination=0.0005,n_jobs=-1,verbose=0)  # Marking 10% as odd
odd_clf.fit(bro_matrix)

IsolationForest(behaviour='new', bootstrap=False, contamination=0.0005,
                max_features=1.0, max_samples='auto', n_estimators=100,
                n_jobs=-1, random_state=None, verbose=0, warm_start=False)

In [21]:
conn_pickle = {'model':odd_clf,'tranformer':to_matrix}

In [22]:
with open('../models/iiesco_iforest_conn_v0.6.pkl', 'wb') as pickle_file:
    joblib.dump(conn_pickle, pickle_file)


In [23]:
# Now we create a new dataframe using the prediction from our classifier
predictions = odd_clf.predict(bro_matrix)

In [24]:
odd_df = df_test[features][predictions == -1]
display_df = df_test[predictions == -1]

In [25]:
# Now we're going to explore our odd observations with help from KMeans
odd_matrix = to_matrix.fit_transform(odd_df)
num_clusters = min(len(odd_df), 4)  # 4 clusters unless we have less than 4 observations
display_df['cluster'] = KMeans(n_clusters=num_clusters).fit_predict(odd_matrix)
print(odd_matrix.shape)


Normalizing column Sport...
Normalizing column udest_port...
Normalizing column netflows...
Normalizing column uproto...
Normalizing column TotDur...
Normalizing column minDur...
Normalizing column maxDur...
Normalizing column tcp...
Normalizing column udp...
Normalizing column internal...
Normalizing column incoming...
Cannot normalize series (div by 0) so not normalizing...
Normalizing column outgoing...
Normalizing column http...
Normalizing column dns...
Normalizing column dhcp...
Cannot normalize series (div by 0) so not normalizing...
Normalizing column ssl...
Normalizing column ssh...
Cannot normalize series (div by 0) so not normalizing...
Normalizing column bytes...
Normalizing column packets...
(1000, 19)


In [26]:
pd.set_option('display.max_rows', 100)
outputdf = pd.DataFrame()
# Now group the dataframe by cluster
# if log_type == 'dns':
#     features += ['query']
#     features+=['ts']  
#     features += ['id.resp_h']

# features += ['id.orig_h']
# features += ['ts']
# features +=['id.resp_h']
# features +=['direction']

In [27]:
display_df =display_df[(display_df['incoming']>0)  | (display_df['outgoing']>0)]
#display(display_df.sort_values(by=['TotDur','tcp','udp','internal','incoming','outgoing','http','dns','dhcp','ssl','ssh','bytes','packets'],ascending=False))
display_df

Sport  udest_port  netflows  uproto   TotDur  minDur  maxDur   tcp   udp  internal  incoming  outgoing  http   dns  dhcp  ssl  ssh      bytes  packets  cluster
SrcAddr      DstAddr        ts                                                                                                                                                                                  
172.16.2.211 8.8.8.8        2020-04-03 04:45:00      1           1        40       1   320.07    8.00    8.01   0.0  40.0       0.0       0.0      40.0   0.0  40.0   0.0  0.0  0.0       8595      200        3
                            2020-04-03 04:50:00      1           1        31       1   248.06    8.00    8.01   0.0  31.0       0.0       0.0      31.0   0.0  31.0   0.0  0.0  0.0       6645      155        1
                            2020-04-03 05:00:00      1           1        31       1   248.04    8.00    8.00   0.0  31.0       0.0       0.0      31.0   0.0  31.0   0.0  0.0  0.0       6690      155        1
                            2020-04-03 05:05:00      1           1        28       1   224.04    8.00    8.01   0.0  28.0       0.0       0.0      28.0   0.0  28.0   0.0  0.0  0.0       6090      140        1
                            2020-04-03 05:10:00      1           1        31       1   248.05    8.00    8.00   0.0  31.0       0.0       0.0      31.0   0.0  31.0   0.0  0.0  0.0       6620      155        1
                            2020-04-03 05:20:00      1           1        29       1   232.05    8.00    8.01   0.0  29.0       0.0       0.0      29.0   0.0  29.0   0.0  0.0  0.0       6210      145        1
172.16.2.90  52.216.129.43  2020-04-03 08:55:00      1           1         1       1   401.77  401.77  401.77   1.0   0.0       0.0       0.0       1.0   0.0   0.0   0.0  1.0  0.0  117317573   128540        2
172.16.2.5   8.8.8.8        2020-03-01 18:40:00      1           1        39       1   299.08    7.00    8.30   0.0  39.0       0.0       0.0      39.0   0.0  39.0   0.0  0.0  0.0       6024      161        3
172.16.2.53  45.60.15.212   2020-03-05 10:30:00      1           1         1       1   887.92  887.92  887.92   1.0   0.0       0.0       0.0       1.0   0.0   0.0   0.0  1.0  0.0    1472273     1831        2
             93.184.221.240 2020-03-05 10:55:00      1           1        34       1  4519.28   95.44  144.29  34.0   0.0       0.0       0.0      34.0  34.0   0.0   0.0  0.0  0.0   24068134    26088        0
                            2020-03-05 11:00:00      1           1        25       1  3363.41  131.45  206.87  25.0   0.0       0.0       0.0      25.0  25.0   0.0   0.0  0.0  0.0  156088401   165306        0
172.16.2.211 8.8.8.8        2020-04-08 13:40:00      1           1        44       1   352.07    8.00    8.01   0.0  44.0       0.0       0.0      44.0   0.0  44.0   0.0  0.0  0.0       9495      220        3
                            2020-04-08 13:50:00      1           1        37       1   296.06    8.00    8.01   0.0  37.0       0.0       0.0      37.0   0.0  37.0   0.0  0.0  0.0       7770      185        3
                            2020-04-08 14:00:00      1           1        27       1   216.05    8.00    8.01   0.0  27.0       0.0       0.0      27.0   0.0  27.0   0.0  0.0  0.0       5850      135        1
                            2020-04-08 14:05:00      1           1        30       1   240.05    8.00    8.01   0.0  30.0       0.0       0.0      30.0   0.0  30.0   0.0  0.0  0.0       6415      150        1
                            2020-04-08 14:15:00      1           1        29       1   232.04    8.00    8.00   0.0  29.0       0.0       0.0      29.0   0.0  29.0   0.0  0.0  0.0       6210      145        1
                            2020-04-08 14:25:00      1           1        32       1   256.05    8.00    8.01   0.0  32.0       0.0       0.0      32.0   0.0  32.0   0.0  0.0  0.0       6825      160        1
                            2020-04-08 14:35:00      1           1        29  

In [28]:
display_df['incoming'].sum()
display_df['outgoing'].sum()
display_df['internal'].sum()

0.0

### Out of two million six hundred sixty-two thousand eight hundred sixty-three records 164 seems anamolous, which are further clustered down based on their similarity 

In [29]:
pd.set_option('display.max_rows', 100)
outputdf = pd.DataFrame()
# Now group the dataframe by cluster
# if log_type == 'dns':
#     features += ['query']
#     features+=['ts']  
#     features += ['id.resp_h']

# features += ['id.orig_h']
# features += ['ts']
# features +=['id.resp_h']
# features +=['direction']

cluster_groups = display_df[features+['cluster']].groupby('cluster')

# Now print out the details for each cluster
print('<<< Outliers Detected! >>>')
for key, group in cluster_groups:
    print('\nCluster {:d}: {:d} observations'.format(key, len(group)))
    group = group[['Sport', 'udest_port', 'netflows', 'uproto', 'TotDur', 'minDur', 'maxDur', 'tcp', 'udp', 'internal', 'incoming', 'outgoing', 'http', 'dns', 'dhcp', 'ssl', 'ssh','bytes','packets']]
    #group = group.T.drop_duplicates().T
    #outputdf = pd.concat([outputdf,group],axis=0)
    group.sort_values(by=['TotDur','tcp','udp','internal','incoming','outgoing','http','dns','dhcp','ssl','ssh','bytes','packets'],ascending=False).to_csv(str(key)+'.csv')
    display(group.sort_values(by=['TotDur','tcp','udp','internal','incoming','outgoing','http','dns','dhcp','ssl','ssh','bytes','packets'],ascending=False))
    

<<< Outliers Detected! >>>

Cluster 0: 2 observations


Sport  udest_port  netflows  uproto   TotDur  minDur  maxDur   tcp  udp  internal  incoming  outgoing  http  dns  dhcp  ssl  ssh      bytes  packets
SrcAddr     DstAddr        ts                                                                                                                                                                       
172.16.2.53 93.184.221.240 2020-03-05 10:55:00      1           1        34       1  4519.28   95.44  144.29  34.0  0.0       0.0       0.0      34.0  34.0  0.0   0.0  0.0  0.0   24068134    26088
                           2020-03-05 11:00:00      1           1        25       1  3363.41  131.45  206.87  25.0  0.0       0.0       0.0      25.0  25.0  0.0   0.0  0.0  0.0  156088401   165306


Cluster 1: 739 observations


Sport  udest_port  netflows  uproto  TotDur  minDur  maxDur  tcp   udp  internal  incoming  outgoing  http   dns  dhcp  ssl  ssh  bytes  packets
SrcAddr      DstAddr ts                                                                                                                                                                   
172.16.2.211 8.8.8.8 2020-04-10 03:35:00      1           1        32       1  256.07     8.0    8.01  0.0  32.0       0.0       0.0      32.0   0.0  32.0   0.0  0.0  0.0   6930      160
                     2020-04-12 05:35:00      1           1        32       1  256.07     8.0    8.01  0.0  32.0       0.0       0.0      32.0   0.0  32.0   0.0  0.0  0.0   6930      160
                     2020-04-13 00:05:00      1           1        32       1  256.07     8.0    8.01  0.0  32.0       0.0       0.0      32.0   0.0  32.0   0.0  0.0  0.0   6895      160
                     2020-04-11 04:30:00      1           1        32       1  256.07     8.0    8.01  0.0  32.0       0.0       0.0      32.0   0.0  32.0   0.0  0.0  0.0   6875      160
                     2020-04-10 23:55:00      1           1        32       1  256.06     8.0    8.01  0.0  32.0       0.0       0.0      32.0   0.0  32.0   0.0  0.0  0.0   6940      160
                     2020-04-08 20:40:00      1           1        32       1  256.06     8.0    8.01  0.0  32.0       0.0       0.0      32.0   0.0  32.0   0.0  0.0  0.0   6930      160
                     2020-04-09 01:55:00      1           1        32       1  256.06     8.0    8.01  0.0  32.0       0.0       0.0      32.0   0.0  32.0   0.0  0.0  0.0   6930      160
                     2020-04-09 12:35:00      1           1        32       1  256.06     8.0    8.01  0.0  32.0       0.0       0.0      32.0   0.0  32.0   0.0  0.0  0.0   6930      160
                     2020-04-10 00:40:00      1           1        32       1  256.06     8.0    8.01  0.0  32.0       0.0       0.0      32.0   0.0  32.0   0.0  0.0  0.0   6930      160
                     2020-04-10 02:10:00      1           1        32       1  256.06     8.0    8.01  0.0  32.0       0.0       0.0      32.0   0.0  32.0   0.0  0.0  0.0   6930      160
                     2020-04-10 16:10:00      1           1        32       1  256.06     8.0    8.01  0.0  32.0       0.0       0.0      32.0   0.0  32.0   0.0  0.0  0.0   6930      160
                     2020-04-10 20:00:00      1           1        32       1  256.06     8.0    8.01  0.0  32.0       0.0       0.0      32.0   0.0  32.0   0.0  0.0  0.0   6930      160
                     2020-04-10 22:40:00      1           1        32       1  256.06     8.0    8.01  0.0  32.0       0.0       0.0      32.0   0.0  32.0   0.0  0.0  0.0   6930      160
                     2020-04-11 14:10:00      1           1        32       1  256.06     8.0    8.01  0.0  32.0       0.0       0.0      32.0   0.0  32.0   0.0  0.0  0.0   6930      160
                     2020-04-11 16:25:00      1           1        32       1  256.06     8.0    8.01  0.0  32.0       0.0       0.0      32.0   0.0  32.0   0.0  0.0  0.0   6930      160
                     2020-04-12 06:25:00      1           1        32       1  256.06     8.0    8.01  0.0  32.0       0.0       0.0      32.0   0.0  32.0   0.0  0.0  0.0   6930      160
                     2020-04-12 20:05:00      1           1        32       1  256.06     8.0    8.01  0.0  32.0       0.0       0.0      32.0   0.0  32.0   0.0  0.0  0.0   6930      160
                     2020-04-13 03:45:00      1           1        32       1  256.06     8.0    8.01  0.0  32.0       0.0       0.0      32.0   0.0  32.0   0.0  0.0  0.0   6930      160
                     2020-04-13 04:25:00      1           1        32       1  256.06     8.0    8.01  0.0  32.0       0.0       0.0      32.0   0.0  32.0   0.0  0.0  0.0   6930      160
                     2020-04-10 06:00:00      1           1        32       1  256.06     8.0    8.01  0.0  32.0   


Cluster 2: 2 observations


,,,Sport,udest_port,netflows,uproto,TotDur,minDur,maxDur,tcp,udp,internal,incoming,outgoing,http,dns,dhcp,ssl,ssh,bytes,packets
SrcAddr,DstAddr,ts,,,,,,,,,,,,,,,,,,,
172.16.2.53,45.60.15.212,2020-03-05 10:30:00,1,1,1,1,887.92,887.92,887.92,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1472273,1831
172.16.2.90,52.216.129.43,2020-04-03 08:55:00,1,1,1,1,401.77,401.77,401.77,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,117317573,128540



Cluster 3: 143 observations


Sport  udest_port  netflows  uproto  TotDur  minDur  maxDur  tcp   udp  internal  incoming  outgoing  http   dns  dhcp  ssl  ssh  bytes  packets
SrcAddr      DstAddr ts                                                                                                                                                                   
172.16.2.211 8.8.8.8 2020-04-09 22:50:00      1           1        50       1  400.08     8.0    8.01  0.0  50.0       0.0       0.0      50.0   0.0  50.0   0.0  0.0  0.0  10350      250
                     2020-04-10 01:50:00      1           1        45       1  360.09     8.0    8.01  0.0  45.0       0.0       0.0      45.0   0.0  45.0   0.0  0.0  0.0   9370      225
                     2020-04-10 22:50:00      1           1        45       1  360.07     8.0    8.01  0.0  45.0       0.0       0.0      45.0   0.0  45.0   0.0  0.0  0.0   9290      225
                     2020-04-08 13:40:00      1           1        44       1  352.07     8.0    8.01  0.0  44.0       0.0       0.0      44.0   0.0  44.0   0.0  0.0  0.0   9495      220
                     2020-04-11 09:50:00      1           1        44       1  352.07     8.0    8.01  0.0  44.0       0.0       0.0      44.0   0.0  44.0   0.0  0.0  0.0   9125      220
                     2020-04-08 22:50:00      1           1        43       1  344.07     8.0    8.01  0.0  43.0       0.0       0.0      43.0   0.0  43.0   0.0  0.0  0.0   8890      215
                     2020-04-09 23:50:00      1           1        43       1  344.06     8.0    8.00  0.0  43.0       0.0       0.0      43.0   0.0  43.0   0.0  0.0  0.0   8970      215
                     2020-04-10 15:50:00      1           1        43       1  344.06     8.0    8.00  0.0  43.0       0.0       0.0      43.0   0.0  43.0   0.0  0.0  0.0   8940      215
                     2020-04-12 18:50:00      1           1        42       1  336.07     8.0    8.01  0.0  42.0       0.0       0.0      42.0   0.0  42.0   0.0  0.0  0.0   8860      210
                     2020-04-09 21:50:00      1           1        42       1  336.07     8.0    8.01  0.0  42.0       0.0       0.0      42.0   0.0  42.0   0.0  0.0  0.0   8755      210
                     2020-04-10 05:50:00      1           1        42       1  336.07     8.0    8.01  0.0  42.0       0.0       0.0      42.0   0.0  42.0   0.0  0.0  0.0   8750      210
                     2020-04-11 20:50:00      1           1        42       1  336.07     8.0    8.01  0.0  42.0       0.0       0.0      42.0   0.0  42.0   0.0  0.0  0.0   8735      210
                     2020-04-10 16:50:00      1           1        42       1  336.07     8.0    8.01  0.0  42.0       0.0       0.0      42.0   0.0  42.0   0.0  0.0  0.0   8715      210
                     2020-04-09 17:50:00      1           1        42       1  336.06     8.0    8.01  0.0  42.0       0.0       0.0      42.0   0.0  42.0   0.0  0.0  0.0   8860      210
                     2020-04-12 03:50:00      1           1        42       1  336.06     8.0    8.00  0.0  42.0       0.0       0.0      42.0   0.0  42.0   0.0  0.0  0.0   8860      210
                     2020-04-11 03:50:00      1           1        42       1  336.06     8.0    8.00  0.0  42.0       0.0       0.0      42.0   0.0  42.0   0.0  0.0  0.0   8765      210
                     2020-04-10 04:50:00      1           1        42       1  336.06     8.0    8.01  0.0  42.0       0.0       0.0      42.0   0.0  42.0   0.0  0.0  0.0   8755      210
                     2020-04-12 07:50:00      1           1        42       1  336.06     8.0    8.01  0.0  42.0       0.0       0.0      42.0   0.0  42.0   0.0  0.0  0.0   8755      210
                     2020-04-10 00:50:00      1           1        41       1  328.08     8.0    8.01  0.0  41.0       0.0       0.0      41.0   0.0  41.0   0.0  0.0  0.0   8550      205
                     2020-04-11 23:50:00      1           1        41       1  328.07     8.0    8.01  0.0  41.0   

In [30]:
display_df[features+['cluster']].groupby('cluster')

<html>
    <center>
        <hr>
        <h1>The End ! </h1>
    </center>
    </html>